<a href="https://colab.research.google.com/github/gunelaliyevaa/wildfire-detection-using-satellite-imagery/blob/main/inceptionv3_train_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TRAINING DEMO NOTEBOOK**

The general purpose of this notebook is to teach the user how to use the InceptionV3 model on a dataset to predict wildfire images in satellite imagery, as part of the user's wildfire detection project.

# **HOW TO SET UP**

 **↪** **Please clone the repository if anything needed to be downloaded:**

In [1]:
!git clone https://github.com/gunelaliyevaa/wildfire-detection-using-satellite-imagery.git

Cloning into 'wildfire-detection-using-satellite-imagery'...
remote: Enumerating objects: 1013, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 1013 (delta 39), reused 47 (delta 36), pack-reused 945 (from 1)
Receiving objects: 100% (1013/1013), 203.09 MiB | 15.30 MiB/s, done.
Resolving deltas: 100% (172/172), done.
Updating files: 100% (678/678), done.


**↪** **Change the working directory to the actual project folder.**

In [2]:
%cd wildfire-detection-using-satellite-imagery

/content/wildfire-detection-using-satellite-imagery


**↪** **Import libraries and install the custom library from the repository**

In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix, classification_report, f1_score, precision_score, recall_score
import tensorflow as tf
import lib as ml

# CONFIGURATION section (optional)

In [ ]:
# Configuration Section

# Dataset Paths
TRAIN_DIR = "/content/drive/MyDrive/sdp_wildfire_satellite_shared/azerbaijan_wildfire_imagery/kaggle_fire_dataset_sample/train_dataset"
TEST_DIR = "/content/drive/MyDrive/sdp_wildfire_satellite_shared/azerbaijan_wildfire_imagery/kaggle_fire_dataset_sample/test_dataset"

# Image Parameters
IMG_SIZE = (350, 350)  # Target size for resizing images
BATCH_SIZE = 32        # Number of images processed in a batch

# Model Training Parameters
EPOCHS = 20            # Initial training epochs
FINE_TUNE_EPOCHS = 10  # Additional epochs for fine-tuning
LEARNING_RATE = 1e-3   # Initial learning rate
FINE_TUNE_LR = 1e-5    # Lower learning rate for fine-tuning

# Data Augmentation Settings
AUGMENTATION_PARAMS = {
    "rescale": 1.0 / 255,
    "rotation_range": 20,
    "width_shift_range": 0.2,
    "height_shift_range": 0.2,
    "shear_range": 0.2,
    "zoom_range": 0.2,
    "horizontal_flip": True,
    "validation_split": 0.2
}

# Callbacks Settings
EARLY_STOPPING_PATIENCE = 3  # Stop training if validation loss doesn't improve
LR_REDUCE_PATIENCE = 2       # Reduce LR if no improvement in val_loss

# Hardware Configuration
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")


# **DATASET PATH, DATA BALANCE CHECK AND VISUALIZATION**

Wildfire Dataset Information is given as:

The dataset consists of satellite images extracted using an image extraction script
from the Sentinel-2 image collection. These images have:
- A resolution of 512x512 pixels
- True-color representation (RGB)
- PNG format

**↪** **We define paths to the wildfire dataset and the dataset contains two categories:**
- 'train-wildfire' - Images showing wildfire scenes.
- 'train-nowildfire' - Images with no wildfire present.

In [ ]:
fire_path = '/content/drive/MyDrive/sdp_wildfire_satellite_shared/azerbaijan_wildfire_imagery/kaggle_fire_dataset_sample/train_dataset/train-wildfire'
nofire_path = '/content/drive/MyDrive/sdp_wildfire_satellite_shared/azerbaijan_wildfire_imagery/kaggle_fire_dataset_sample/train_dataset/train-nowildfire'

**↪** **We count the number of images in each category ensuring dataset is balanced since this is crucial for accuracy of the trained model. Printing results, we get count of each class.**

In [ ]:
fire_images = len(os.listdir(fire_path))
nofire_images = len(os.listdir(nofire_path))

print(f"Fire Images: {fire_images}")
print(f"No Fire Images: {nofire_images}")

**↪** **We can visualize a batch of fire and nofire images below**

In [ ]:
# Get a batch of images from the training generator
fire_batch, fire_labels = next(train_generator)

# Function to display a batch of images
def visualize_batch(images, labels, class_names, title):
    plt.figure(figsize=(10, 10))
    for i in range(9):  # Display 9 images
        plt.subplot(3, 3, i+1)
        plt.imshow(images[i])  # Images are already normalized
        label = class_names[int(labels[i])]  # Convert label to class name
        plt.title(label)
        plt.axis("off")
    plt.suptitle(title, fontsize=14)
    plt.show()

# Class labels based on binary classification
class_names = {0: "No Fire", 1: "Fire"}

# Display batch of images
visualize_batch(fire_batch, fire_labels, class_names, "Sample Fire and No Fire Images")


# **DATA PREPROCESSING**

**Data Augmentation & Preprocessing**

**↪** **The dataset will be preprocessed and augmented using ImageDataGenerator.
This helps improve model generalization by introducing slight variations in the images. Augmentations applied:**
- Rescaling pixel values to the range [0,1] for normalization ( ❓ Neural networks perform better when input values are normalized. Rescaling helps speed up training and stabilizes weight updates.)
- Random rotations (up to 20 degrees) ( ❓ Wildfire images may be captured from different angles due to satellite positioning. Rotations help the model learn orientation-invariant features.)
- Horizontal shifting (up to 20% of image width) ( ❓ Fires may appear at different locations within an image. This shift prevents the model from over-relying on fixed spatial positions.)
- Vertical shifting (up to 20% of image height) ( ❓ Similar to horizontal shifts, vertical shifts ensure the model does not depend on absolute positions of fire regions.)
- Shearing transformations (up to 20%) ( ❓ Introduces a subtle distortion effect, mimicking real-world variations caused by different camera angles or satellite distortions.)
- Zooming (up to 20%) ( ❓ Helps the model recognize fires at different scales, ensuring it can detect both small and large fire regions.)
- Horizontal flipping ( ❓ Since fires can occur in any direction, flipping prevents the model from favoring a specific orientation.)
- Splitting into training (80%) and validation (20%) sets ( ❓ Ensures the model is trained on one subset of the data while another unseen subset is used to evaluate its performance, preventing overfitting.)

In [ ]:
datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Split
)

# **LOAD DATA FROM DIRECTORIES**


**The generator loads images from the dataset directory, applies transformations,and prepares them for model training:**

In [ ]:
train_generator = datagen.flow_from_directory(
    '/content/drive/MyDrive/sdp_wildfire_satellite_shared/azerbaijan_wildfire_imagery/kaggle_fire_dataset_sample/train_dataset',
    target_size=(350, 350),  # Resize all images to 350x350
    batch_size=32,  # Process images in mini-batches of 32
    class_mode='binary',  # Binary classification: wildfire vs. no wildfire
    subset='training'  # Load only training images (80% of total)
)

**This generator loads the validation images (20% of the dataset) for model evaluation.**

In [ ]:
val_generator = datagen.flow_from_directory(
    '/content/drive/MyDrive/sdp_wildfire_satellite_shared/azerbaijan_wildfire_imagery/kaggle_fire_dataset_sample/train_dataset',
    target_size=(350, 350),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# **A BIT ABOUT MODEL**

> The wildfire classification model is based on the InceptionV3 architecture,
> a well-known deep convolutional neural network originally designed for image recognition.
> InceptionV3 is pre-trained on ImageNet and fine-tuned on our wildfire dataset
> to differentiate between wildfire and non-wildfire images.
> The model is capable of extracting deep spatial features from images while  maintaining computational efficiency. It employs factorized convolutions and  auxiliary classifiers to improve learning performance and reduce overfitting.

>**InceptionV3 Model Structure**
>The InceptionV3 model is initialized with pre-trained weights from ImageNet and adapted for binary classification: wildfire vs. no wildfire.

>The original fully connected layers are replaced with a custom classifier
>The input image size is set to (350x350), consistent with the dataset
>The model applies a global average pooling layer before classification
>The final activation function is sigmoid, as this is a binary classification task

>While the base InceptionV3 model is powerful, additional techniques such as
data augmentation and transfer learning help improve performance.
>Fine-tuning deeper layers of the model allows it to learn wildfire-specific patterns rather than generic image features.

>The model is trained using binary cross-entropy loss and Adam optimizer,
with real-time data augmentation applied to enhance generalization.

# **Model Architecture and Compilation**

The wildfire classification model is built using InceptionV3 as a feature extractor.

This approach leverages the powerful convolutional layers of InceptionV3,
which have been pre-trained on the large-scale ImageNet dataset.
Instead of training from scratch, we use transfer learning by freezing
the base layers and adding a custom classifier on top.
This allows the model to retain general image features while focusing
on wildfire-specific patterns in the dataset.

**↪** **Load Pretrained InceptionV3 Model:**
The model is initialized with pre-trained ImageNet weights.
The top (fully connected) layers are removed to allow customization.
The input size is set to (350x350x3), matching the dataset resolution.

In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(350, 350, 3))

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


 **↪** **Freeze Base Model Layers:** Freezing prevents the pre-trained convolutional layers from being updated. This ensures the model retains the general feature extraction capabilities.Only the newly added classifier layers will be trained

In [ ]:
base_model.trainable = False

NameError: name 'base_model' is not defined

**↪** **Define New Model:**
A Flatten layer converts extracted features into a 1D vector
A fully connected Dense layer with 256 neurons and ReLU activation is added
The final output layer has a single neuron with sigmoid activation for binary classification

In [ ]:
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification: wildfire vs. no wildfire
])

**↪** **Compile Model**
Adam optimizer is used for efficient training
Binary cross-entropy is the loss function, suitable for binary classification
Accuracy is used as the evaluation metric

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# **Training the Model**

The model is trained using augmented wildfire images, with validation data used to monitor performance. Early stopping and learning rate adjustments are applied to optimize training efficiency.

 **↪** **Define Callbacks:**
- EarlyStopping: Stops training if validation loss does not improve for 3 epochs, restoring the best model weights to prevent overfitting ReduceLROnPlateau: Reduces learning rate by a factor of 0.5 if validation loss
stagnates for 2 epochs, ensuring better convergence.

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

**↪** **Train the Model:**
 - The model is trained for 20 epochs using the preprocessed dataset. Training data is fed using an augmented ImageDataGenerator. The validation dataset is used to evaluate generalization. Callbacks ensure efficient training by preventing overfitting and adjusting learning rates.

In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[early_stopping, lr_scheduler]
)

 **↪** **Plot Training and Validation Accuracy:**
- Accuracy curves are plotted to visualize model performance.
The trend helps identify overfitting or underfitting issues.

In [ ]:
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()

# **FINE TUNING THE MODEL**

 **↪** **Unfreeze base model layers for further training. Use a lower learning rate for gradual adjustments**

In [ ]:
base_model.trainable = True

# Recompile with lower LR
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=config['lr']),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Fine-tune for 10 epochs
history_ft = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[early_stopping, lr_scheduler]
)

# Plot accuracy
plt.plot(history_ft.history['accuracy'], label='Training Accuracy (Fine-tune)')
plt.plot(history_ft.history['val_accuracy'], label='Validation Accuracy (Fine-tune)')
plt.legend()
plt.title('Fine-Tuning Accuracy')
plt.show()